<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Kaggle-Challenge/blob/master/Kaggle_Challenge_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install --upgrade category_encoders plotly
!pip install fiona
!pip install geopandas

In [3]:
import os, sys

os.chdir('/content')
!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
!git pull origin master
    
# Install required python packages
!pip install -r requirements.txt
    
# Change into directory for module
os.chdir('module1')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), done.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 112kB 2.9MB/s 
     |████████████████████████████████| 133kB 9.0MB/s 
     |████████████████████████████████| 57.7MB 1.2MB/s 
     |████████████████████████████████| 245kB 52.3MB/s 
     |████████████████████████████████| 614kB 44.5MB/s 
     |████████████████████████████████| 225kB 41.7MB/s 
     |████████████████████████████████| 768kB 47.7MB/s 
     |████████████████████████████████| 51kB 25.4MB/s 
     |████████████████████████████████| 194kB 45.9MB/s 
     |████████████████████████████████| 51kB 21.0MB/s 
     |████████████████████████████████| 737kB 52.0MB/s 
  Created wheel for pandas-profiling:

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [0]:
import numpy as np

# Wrangle train, validate, and test sets in the same way
def wrangle(X):
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Clean installer
    X['installer'] = X['installer'].str.lower()
    X['installer'] = X['installer'].str.replace('danid', 'danida')
    X['installer'] = X['installer'].str.replace('disti', 'district council')
    X['installer'] = X['installer'].str.replace('commu', 'community')
    X['installer'] = X['installer'].str.replace('central government', 'government')
    X['installer'] = X['installer'].str.replace('kkkt _ konde and dwe', 'kkkt')
    X['installer'].value_counts(normalize=True)
    tops = X['installer'].value_counts()[:5].index
    X.loc[~X['installer'].isin(tops), 'installer'] = 'Other'
    
    # Clean funder and bin
    X['funder'] = X['funder'].str.lower()
    X['funder'] = X['funder'].str[:3]
    X['funder'].value_counts(normalize=True)
    tops = X['funder'].value_counts()[:20].index
    X.loc[~X['funder'].isin(tops), 'funder'] = 'Other'

    # Use mean for gps_height missing values
    X.loc[X['gps_height'] == 0, 'gps_height'] = X['gps_height'].mean()
    
    # Bin lga
    X['lga'].value_counts(normalize=True)
    tops = X['lga'].value_counts()[:10].index
    X.loc[~X_train['lga'].isin(tops), 'lga'] = 'Other'

    # Bin ward 
    X['ward'].value_counts(normalize=True)
    tops = X['ward'].value_counts()[:20].index
    X.loc[~X['ward'].isin(tops), 'ward'] = 'Other'

    # Clean latitude and longitude
    average_lat = X.groupby('region').latitude.mean().reset_index()
    average_long = X.groupby('region').longitude.mean().reset_index()

    shinyanga_lat = average_lat.loc[average_lat['region'] == 'Shinyanga', 'latitude']
    shinyanga_long = average_long.loc[average_lat['region'] == 'Shinyanga', 'longitude']

    X.loc[(X['region'] == 'Shinyanga') & (X['latitude'] > -1), ['latitude']] = shinyanga_lat[17]
    X.loc[(X['region'] == 'Shinyanga') & (X['longitude'] == 0), ['longitude']] = shinyanga_long[17]

    mwanza_lat = average_lat.loc[average_lat['region'] == 'Mwanza', 'latitude']
    mwanza_long = average_long.loc[average_lat['region'] == 'Mwanza', 'longitude']

    X.loc[(X['region'] == 'Mwanza') & (X['latitude'] > -1), ['latitude']] = mwanza_lat[13]
    X.loc[(X['region'] == 'Mwanza') & (X['longitude'] == 0) , ['longitude']] = mwanza_long[13]
    
    # Impute mean for tsh based on mean of source_class/basin/waterpoint_type_group
    def tsh_calc(tsh, source, base, waterpoint):
      if tsh == 0:
        if (source, base, waterpoint) in tsh_dict:
          new_tsh = tsh_dict[source, base, waterpoint]
          return new_tsh
      else:
        return tsh
    return tsh
  
    temp = X[X['amount_tsh'] != 0].groupby(['source_class',
                                            'basin',
                                            'waterpoint_type_group'])['amount_tsh'].mean()

    tsh_dict = dict(temp)
    X['amount_tsh'] = X.apply(lambda x: tsh_calc(x['amount_tsh'], x['source_class'], x['basin'], x['waterpoint_type_group']), axis=1)
               
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns='quantity_group')
    
    # return the wrangled dataframe
    return X




In [0]:
#import plotly.express as px
#px.scatter(train, x='longitude', y='latitude', color='status_group', opacity=0.1)

In [0]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [7]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target & id
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)


['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]